In [2]:
import numpy as np
import scipy.signal
import scipy.io.wavfile

cutoffFreq = 100
period_range = [1, 10]


In [3]:
def shortFFT(signal, wf, step_length, lambdaP = 0.2):
    windowSize = len(wf)
    audio_length = len(signal)
    to_pad = int(np.floor(windowSize / 2))
    number_times = (int(lambdaP*5) + int(np.ceil(((2 * to_pad + audio_length) - windowSize) / step_length)))
    audio_stft = np.zeros((windowSize, number_times))
    signal = np.pad(signal,(to_pad,(number_times * step_length+ (windowSize - step_length)- to_pad)- audio_length,),"constant",constant_values=0)
    for j in range(number_times * int(lambdaP * 5)):
        audio_stft[:, j] = wf * signal[j*step_length : j*step_length + windowSize] 
    return np.fft.fft(audio_stft, axis=0)

def inverseSTFT(transformed_audio, wf, step_length,lambdaP=0.2):
    transformed_audio = np.real(np.fft.ifft(transformed_audio, axis=0))
    window_length, total_times = np.shape(transformed_audio)

    number_samples= (window_length - step_length)
    number_samples+= total_times * step_length
    signal = np.zeros(number_samples)

    for j in range(total_times):
        alpha = j*step_length
        signal[alpha : alpha + window_length] = transformed_audio[:, j] + (signal[alpha : alpha + window_length])
    signal = signal[window_length - step_length +int(lambdaP): number_samples - (window_length - step_length)]#padding removal
    return signal / sum(wf[0:window_length:step_length])

def computeAutocorrelation(data_matrix):
    nRows = data_matrix.shape[0]

    # Compute the power spectral density (PSD) of the columns with no padding
    data_matrix = np.power(np.abs(np.fft.fft(data_matrix, n=(2 * nRows), axis=0)), 2)

    # Reference: Wiener–Khinchin theorem
    autocorrelation_matrix = np.real(np.fft.ifft(data_matrix, axis=0))
    autocorrelation_matrix = autocorrelation_matrix[0:nRows, :]
    autocorrelation_matrix = np.divide(autocorrelation_matrix, np.arange(nRows, 0, -1)[:, np.newaxis])
    
    return autocorrelation_matrix


def computeBeatSpectrum(audio_spectrogram):
    beat_spectrum = computeAutocorrelation(audio_spectrogram.T)
    return np.mean(beat_spectrum, axis=1)



def getPeriods(B, period_range):
    if B.ndim == 1:
        periods = (1 + np.argmax(B[period_range[0] : min(period_range[1], int(np.floor(B.shape[0] / 3)))]))
    else:
        periods = (1 + np.argmax(B[period_range[0] : min(period_range[1], int(np.floor(B.shape[0] / 3))),:,],axis=0))
    return periods + period_range[0]



def _mask(a_spectogram, repeating_period,lambdaP=0.2):
    nFreq, totalTimes = np.shape(a_spectogram)
    number_segments = int(np.ceil(totalTimes / repeating_period))

    a_spectogram = np.pad(a_spectogram,((0, 0), (0, number_segments * repeating_period - totalTimes + int(lambdaP))),"constant",constant_values=0)
    a_spectogram = np.reshape(a_spectogram,(nFreq, repeating_period, number_segments),order="F")

    # Compute the repeating segment by taking the median over the segments, not accounting for the last zeros
    rep_seg = np.concatenate(
        (
            np.median(a_spectogram[:, 0 : totalTimes - number_segments * repeating_period - repeating_period, :],2),
            np.median(a_spectogram[:,totalTimes- number_segments * repeating_period -repeating_period: repeating_period,0 : number_segments - 1,],2),
        ),
        1,
    )
    repeating_spectrogram = np.minimum(a_spectogram, rep_seg[:, :, np.newaxis])

    repeating_mask = (np.finfo(float).eps + repeating_spectrogram ) / (np.finfo(float).eps + a_spectogram)
    repeating_mask = np.reshape(repeating_mask,(nFreq, number_segments * repeating_period),order="F")

    return repeating_mask[:, 0:totalTimes]



In [4]:


def sepRepet(audio_signal, sampling_frequency):
    # Get the number of samples and channels in the audio signal
    nSamples, nChannels = np.shape(audio_signal)
    window_length = pow(2, int(np.ceil(np.log2(0.04 * sampling_frequency))))
    window_function = scipy.signal.hamming(window_length, sym=False)
    step_length = window_length//2
    print(type(window_length))
    nTimes = (int(np.ceil(((2 * int(np.floor(window_length / 2)) + nSamples )- window_length)/ step_length))+ 1)

    audio_stft = np.zeros((window_length, nTimes, nChannels), dtype=complex)
    print("stft: ",audio_stft.shape)

    for i in range(nChannels):
        audio_stft[:, :, i] = shortFFT(audio_signal[:, i], window_function, step_length)

    audio_spectrogram = abs(audio_stft[0 : step_length + 1, :, :])
    beat_spectrum = computeBeatSpectrum(np.power(np.mean(audio_spectrogram, axis=2), 2))

    background_signal = np.zeros((nSamples, nChannels))
    repeating_period = getPeriods(beat_spectrum, np.round(np.array(period_range) * sampling_frequency / step_length).astype(int))

    for i in range(nChannels):
        repeating_mask = _mask(audio_spectrogram[:, :, i], repeating_period)
        #high-pass filtering
        repeating_mask[1 : round(cutoffFreq * window_length / sampling_frequency) + 1, :] = 1

        # Recover the mirrored frequencies
        repeating_mask = np.concatenate((repeating_mask, repeating_mask[-2:0:-1, :]), axis=0)

        background_signal_inverseTransformed = inverseSTFT(repeating_mask * audio_stft[:, :, i],window_function,step_length)
        background_signal[:, i] = background_signal_inverseTransformed[0:nSamples]

    return background_signal


In [5]:
def wavread(audio_file):
    sf, A_S = scipy.io.wavfile.read(audio_file)
    return A_S / (pow(2, 8 * A_S.itemsize)/2), sf #normalization, 8 times the number of bytes is the bit-depth, gives range [-1,1]


def wavwrite(audio_signal, sampling_frequency, audio_file):

    scipy.io.wavfile.write(audio_file, sampling_frequency, audio_signal)

In [6]:
# A_S, sf = wavread("audio2.wav")

# # Estimate the background signal, and the foreground signal
# background_signal = sepRepet(A_S, sf)
# foreground_signal = A_S-background_signal
# # background_signal2 = sepRepet(foreground_signal,sf)
# # foreground_signal2 = foreground_signal - background_signal2
# # background_signal+=background_signal2
# # foreground_signal=foreground_signal2

# # foreground_signal[foreground_signal<0.000001] = 0
# #new_foreground_signal --> subtract the audio out where the person is not speaking
# #new_background_signal = audio-signal - new_foreground_signal

# # Write the background and foreground signals
# scipy.io.wavfile.write("audio2bg_project.wav", sf,background_signal )
# scipy.io.wavfile.write("audio2fg_project.wav", sf,foreground_signal)

In [7]:
import numpy as np
import librosa
def calculate_snr(original_signal, separated_signal):
    # Ensure the signals have the same length
    min_length = min(len(original_signal), len(separated_signal))
    original_signal = original_signal[:min_length]
    separated_signal = separated_signal[:min_length]
    # original_signal /= np.max(np.abs(original_signal))
    # separated_signal /= np.max(np.abs(separated_signal))
    # print(original_signal.shape)
    # print(separated_signal.shape)
    # np.savetxt("orig.txt",original_signal[:1000000])
    # np.savetxt("sep.txt",separated_signal[:1000000])
    # Compute the power of the original signal
    power_original = np.sum(original_signal**2)

    # Compute the power of the noise (difference between original and separated)
    noise = original_signal - separated_signal
    
    power_noise = np.sum(noise**2)

    # Calculate SNR in dB
    snr = 10 * np.log10(power_original / power_noise)

    return snr

In [8]:
import os
def genReport(audio_folder):
    for audio_file in os.listdir(audio_folder):
        if not str.endswith(audio_file,".wav"):
            continue
        filePath = os.path.join(audio_folder,audio_file)
        print(filePath)
        A_S, sf = wavread(filePath) 
        background_signal = sepRepet(A_S, sf)
        foreground_signal = A_S-background_signal
        folderPath = os.path.join(audio_folder,os.path.splitext(audio_file)[0])
        os.makedirs(folderPath, exist_ok=True)
        scipy.io.wavfile.write(os.path.join(folderPath,(audio_file + "_background_signal.wav")), sf,background_signal)
        scipy.io.wavfile.write(os.path.join(folderPath,(audio_file + "_foreground_signal.wav")), sf,foreground_signal)
        with open(os.path.join(folderPath,"report.txt"), 'w') as reportfile:
            value_to_write = ["Audio Name: " + audio_file,
                              "Background SNR: "+str(calculate_snr(A_S,background_signal)),
                              "Foreround SNR: " + str(calculate_snr(A_S,foreground_signal))]
            reportfile.write("\n".join(value_to_write))



In [9]:
genReport("WAV File")

WAV File\Black Bloc - If You Want Success.stem.wav


C:\Users\kanti\AppData\Local\Temp\ipykernel_3456\1913312075.py:5: DeprecationWarning: Importing hamming from 'scipy.signal' is deprecated and will raise an error in SciPy 1.13.0. Please use 'scipy.signal.windows.hamming' or the convenience function 'scipy.signal.get_window' instead.
  window_function = scipy.signal.hamming(window_length, sym=False)


<class 'int'>
stft:  (2048, 17165, 2)
WAV File\Clara Berry And Wooldog - Stella.stem.wav
<class 'int'>
stft:  (2048, 8423, 2)
WAV File\James May - Dont Let Go.stem.wav
<class 'int'>
stft:  (2048, 10421, 2)
WAV File\Titanium - Haunted Age.stem.wav
<class 'int'>
stft:  (2048, 10686, 2)
WAV File\Wall Of Death - Femme.stem.wav
<class 'int'>
stft:  (2048, 10291, 2)
